In [1]:
# Required Lib

import random
import math
from math import log, exp, e
from datetime import datetime, timedelta
import unicodedata       

In [2]:
# USER & PRODUCT Backend Logic
# Product

def product_age_adjust(days_since_creation: int):
  age_adjustment = 3 -(days_since_creation) / 25
  return max(age_adjustment,-10)

def product_inventory_adjust(avg_inventory: int, min_limit=-5):
    if avg_inventory < 0:
        avg_inventory = 0
    adjustment = 15 - (5 * log(avg_inventory + 1))
    return max(adjustment, min_limit)

# def product_inventory_adjust(avg_inventory: int):
#   inventory_adjustment = 15 - (5 * log(avg_inventory + 1))
#   return max(inventory_adjustment,-5)

def produt_image_adjust(image_count: int):
  image_bonus = 2 * min(image_count, 5)
  return image_bonus

def product_type_adjust(product_type):
  if product_type == 'seasonal':
    product_type_adjust = 5
  else:
    product_type_adjust = 0
  return product_type_adjust

seasonal_tags = ["winter", "summer", "spring", "autumn"]
def product_tag_adjust(product_tag):
  if product_tag in seasonal_tags:
    product_tag_adjust = 5
  else:
    product_tag_adjust = 0
  return product_tag_adjust

def avg_price_adjust(avg_price: float):
  price_adjustment = min(15,3 * log(avg_price + 1) - 10)
  return price_adjustment


def calculate_product_score(days_since_creation , avg_inventory, image_count, product_type, product_tag, avg_price):
  age_adjustment = product_age_adjust(days_since_creation )
  inventory_adjustment = product_inventory_adjust(avg_inventory)
  image_bonus = produt_image_adjust(image_count)
  product_type = product_type_adjust(product_type)
  product_tag = product_tag_adjust(product_tag)
  price_adjustment = avg_price_adjust(avg_price)
  return max(age_adjustment + inventory_adjustment + image_bonus + product_type + product_tag + price_adjustment,0) 


# User score 

def user_order_score(order_count):
  order_score =min(15, 3 *log(order_count + 1))
  return order_score

# replace with user_cancelled_order_penalty
def user_refund_penalty(total_refunded, total_spent):
  refund_penalty = -10 * (total_refunded / (total_spent + 1))
  return refund_penalty

def user_recency_bonus(days_since_last_purchase):
  recency_bonus = 10 * exp(-days_since_last_purchase / 100)
  return recency_bonus

def user_spending_adjust(total_spent):
  return 15 * (2 / (1 + exp(-0.002 * (total_spent - 500))) - 1)

def user_email_verify_adjust(verified_email):
  if verified_email:
        return 5
  else:
    return 0


def calculate_user_score(order_count, total_refunded, total_spent, days_since_last_purchase, verified_email):
  order_score = user_order_score(order_count)
  refund_penalty = user_refund_penalty(total_refunded, total_spent)
  recency_bonus = user_recency_bonus(days_since_last_purchase)
  spending = user_spending_adjust(total_spent)
  verify_email = user_email_verify_adjust(verified_email)
  return order_score + refund_penalty + recency_bonus + spending + verify_email

In [3]:
# Max round calculation Logic

MAX_USER_SCORE = 45
MAX_PRODUCT_SCORE = 53
MAX_BARGAIN_SCORE = 100

def calculate_max_rounds(user_score, product_score, bargain_score):
  # Normalize user and bargain scores between 0 and 1
  user_factor = user_score / MAX_USER_SCORE
  bargain_factor = bargain_score / MAX_BARGAIN_SCORE
  # Use an inverse relation for product influence (higher product score reduces rounds)
  product_factor = 1 / (product_score + 1)  # Avoid division by zero
  max_rounds = min(6, round((user_factor + product_factor + bargain_factor) * 3) + 2)

  return max_rounds

In [4]:
# Max allowed discount based on different scores 

def adjust_max_discount (admin_max_discount, user_score, product_score,bargain_score, max_user_score=45, max_product_score=53):
  # Normalize user influence (range: 0 to 1)
  user_factor = user_score / max_user_score
  # Scale product influence (higher product score reduces discount, max 30% reduction)
  product_factor = 1 - (product_score / max_product_score) * 0.3
  # Adjust bargain factor with a 30% effect
  normalized_bargain_score = max(0.5, min(1, bargain_score / 65))
  bargain_factor = 1 - 0.3 * (1 - normalized_bargain_score)  # Only 30% effect
  adjusted_discount = admin_max_discount * user_factor * product_factor * bargain_factor
  adjusted_discount = math.ceil(adjusted_discount)
    
  return max(adjusted_discount, 0.5*admin_max_discount) #ensures that discount never goes zero

In [5]:
# Get discounts for all rounds

def calculate_round_discount(max_round, round_count, max_disc):
    if round_count < 0 or round_count > max_round:
        raise ValueError("round_count must be between 0 and max_round")

    # Ensure a smooth logarithmic increase, avoiding zero at round_count=0
    base_discount = max_disc * 0.1  # 10% of max discount as a base
    scaling_factor = log(round_count + 1) / log(max_round + 1)

    # Calculate discount dynamically, ensuring it starts small but > 0
    discount = base_discount + (scaling_factor * (max_disc - base_discount))

    return math.floor(discount)  # Round down to the nearest integer

def generate_all_round_discounts(max_round, max_disc):
    round_discounts = {}

    for round_count in range(1, max_round + 1):
        discount = calculate_round_discount(max_round, round_count, max_disc)
        round_discounts[f'round_count{round_count}'] = discount  # Key format

    return round_discounts

In [6]:
# Handle User Email and Product details

def get_product_detail(product_name, product_descriptions, product_sizes, color):
    # Get the selected product description
    description = product_descriptions.get(product_name) #, "Description not available")
    # Get the available sizes for the selected product
    sizes = product_sizes.get(product_name) #, ["One Size"])
    # Select a random size from the available sizes
    selected_size = random.choice(sizes)
    # Get the selected random color
    product_color = color
    
    # Output the selected product and size
    return {
        "Product_Name": product_name,
        "Product_Description": description,
        "Product_Size": selected_size,
        "Product_color": product_color
}


def user_email_handler(user_name):
    if user_name:
        user_email = f"{user_name.lower()}@gmail.com"
    else:
        user_email = user_name

    return user_email

In [7]:
# Admin Minimum Price Handler

def get_min_price_from_product_price(product_price):
    max_discount_percentage = random.randint(10, 60)
    #print(f"max_discount_percentage : {max_discount_percentage}")
    discount_amount = product_price * (max_discount_percentage / 100)
    #print(f"discount_amount:{discount_amount}")

    min_price = round(product_price - discount_amount)
    #print(f"min_price:{min_price}")
    return min_price, max_discount_percentage

In [8]:
# Current round discounted number
def get_discount_for_current_round(current_round_count, discount_round_count):
    # Construct the key name dynamically
    key = f'round_count{current_round_count}'
    # Get the corresponding discount
    current_discount = discount_round_count.get(key)
    return current_discount


# Get price for the discounted number
def get_original_price_from_discounts(Original_Price, Discount_Percentage):
    Discounted_original_Price = Original_Price * (1 - Discount_Percentage / 100)
    return Discounted_original_Price 

# Get previous round discount and user re for scenario gen
def get_previous_round(current_round, round_counts_with_discounts):
    previous_round_key = f'round_count{current_round - 1}'
    previous_round_discount = round_counts_with_discounts.get(previous_round_key, None)
    user_add = random.randint(9, 29)
    #print(user_add)
    user_req = previous_round_discount + user_add
    return previous_round_discount, user_req

In [9]:
# To get round count between 3 to 6 

def get_lower_bound(max_round_count):
    if max_round_count <= 3:
        lower_bound = 3
    elif max_round_count >= 4:
        lower_bound = 4
    return lower_bound

In [10]:
# for generating clean dataset

def normalize_text(text):
    if isinstance(text, str):
        return unicodedata.normalize("NFKD", text).encode("ascii", "ignore").decode("utf-8")
    return text  # Return as-is if not a string


# # Normalize key text fields
# user = normalize_text(user)
# response = normalize_text(response)
# Data_from_data_base = normalize_text(Data_from_data_base)

# Use standard json.dumps — no need for ensure_ascii=False 
# or
# print(json.dumps(dataset, indent=2, ensure_ascii=False))

In [14]:
# Main Function

def generate_scenario():
    
    user_name = random.choice([
    "Ramesh", "Sneha", "Aman", "Priya", "Nikhil", "Ayesha", "Zara", "Kiran", "Manav", "Diya",
    "Arjun", "Ananya", "Ishaan", "Kiara", "Neha", "Aarav", "Meera", "Vivaan", "Saanvi", "Devika",
    "Rahul", "Anika", "Dhruv", "Kavya", "Riya", "Aarohi", "Tanvi", "Yashvi", "Eshaan", "Shanaya",
    "Manish", "Avani", "Bhavya", "Chetan", "Deepak", "Anjali", "Girish", "Pooja", "Vidhi", "", None])
    
    user_email = user_email_handler(user_name)
    
    product_name = random.choice([
    "Shirt", "Shoes", "Watch", "Jacket", "Backpack", "Sunglasses", "Sweater", "Kurta", "Jeans", "T-shirt",
    "Scarf", "Hat", "Sneakers", "Boots", "Blazer", "Wallet", "Gloves", "Belt", "Tie", "Cufflinks",
    "Chinos", "Cardigan", "Pajamas", "Shorts", "Raincoat", "Sweatshirt", "Tracksuit", "Vest", "Overcoat",
    "Sandals", "Flip-flops", "Beanie", "Headband", "Socks", "Umbrella", "Luggage", "Duffel Bag", "Laptop Sleeve",
        # Electronic
    "Smartphone", "Laptop", "Tablet", "Smartwatch", "Bluetooth Headphones", "Wireless Earbuds", "Smart TV", "Digital Camera", "Portable Speaker", "Power Bank",
    "Electric Kettle", "Hair Dryer", "Electric Toothbrush", "Electric Shaver", "Microwave Oven", "Refrigerator", "Washing Machine", "Air Conditioner", "Water Purifier", "Electric Fan",
    "Induction Cooktop", "Electric Pressure Cooker", "Electric Grill", "Electric Rice Cooker", "Electric Juicer", "Electric Mixer Grinder", "Electric Iron", "Electric Heater", "Electric Blanket", "Electric Stove",
    "Electric Oven", "Electric Chimney", "Electric Geyser", "Electric Water Heater", "Electric Cooker", "Electric Deep Fryer", "Electric Sandwich Maker", "Electric Popcorn Maker", "Electric Coffee Maker", "Electric Tea Maker", "Electric Milk Frother"])

    product_descriptions = {
    "Shirt": "A classic button-down shirt made from breathable cotton, perfect for both casual and formal occasions.",
    "Shoes": "Stylish leather shoes that offer comfort and durability, ideal for daily wear.",
    "Watch": "A sleek stainless steel watch with a minimalist design, featuring water resistance and a date function.",
    "Jacket": "A lightweight bomber jacket with ribbed cuffs and hem, suitable for layering in cooler weather.",
    "Backpack": "Spacious and ergonomic backpack with multiple compartments, designed for daily commuting.",
    "Sunglasses": "UV-protective sunglasses with polarized lenses, combining fashion and functionality.",
    "Sweater": "Soft woolen sweater with a cozy fit, perfect for chilly evenings.",
    "Kurta": "Traditional Indian kurta made from cotton, featuring intricate embroidery and a relaxed fit.",
    "Jeans": "Denim jeans with a slim fit and stretchable fabric, offering both style and comfort.",
    "T-shirt": "Cotton T-shirt with a graphic print, available in various colors.",
    "Scarf": "Lightweight scarf made from silk, adding a touch of elegance to any outfit.",
    "Hat": "Wide-brimmed hat providing sun protection and a stylish look.",
    "Sneakers": "Comfortable sneakers with cushioned soles, suitable for running and casual outings.",
    "Boots": "Leather boots with a rugged design, perfect for outdoor adventures.",
    "Blazer": "Tailored blazer with a modern cut, ideal for professional settings.",
    "Wallet": "Genuine leather wallet with multiple card slots and a coin pocket.",
    "Gloves": "Thermal gloves designed to keep your hands warm during winter.",
    "Belt": "Classic leather belt with a polished buckle, complementing formal attire.",
    "Tie": "Silk tie with a subtle pattern, adding sophistication to your outfit.",
    "Cufflinks": "Elegant cufflinks crafted from sterling silver, enhancing your shirt cuffs.",
    "Chinos": "Cotton chinos with a slim fit, versatile for both work and casual wear.",
    "Cardigan": "Button-up cardigan made from soft cotton, perfect for layering.",
    "Pajamas": "Comfortable cotton pajamas set, ensuring a good night's sleep.",
    "Shorts": "Casual shorts with an adjustable waistband, suitable for warm weather.",
    "Raincoat": "Waterproof raincoat with a hood, keeping you dry during showers.",
    "Sweatshirt": "Cotton sweatshirt with a fleece lining, offering warmth and comfort.",
    "Tracksuit": "Matching tracksuit with zippered pockets, ideal for workouts.",
    "Vest": "Lightweight vest with a zip closure, suitable for layering.",
    "Overcoat": "Woolen overcoat with a double-breasted design, perfect for winter.",
    "Sandals": "Comfortable sandals with adjustable straps, suitable for summer outings.",
    "Flip-flops": "Rubber flip-flops with a cushioned footbed, ideal for beach trips.",
    "Beanie": "Knit beanie hat, keeping your head warm during cold weather.",
    "Headband": "Stretchable headband, perfect for workouts or casual wear.",
    "Socks": "Cotton socks with reinforced heels and toes, ensuring durability.",
    "Umbrella": "Compact umbrella with a wind-resistant frame, easy to carry.",
    "Luggage": "Hard-shell luggage with spinner wheels, offering smooth mobility.",
    "Duffel Bag": "Spacious duffel bag with a detachable shoulder strap, suitable for travel.",
    "Laptop Sleeve": "Padded laptop sleeve with a zipper closure, protecting your device.",
     # Electronic   
    "Smartphone": "A sleek smartphone with a high-resolution camera, long-lasting battery, and a fast processor, perfect for everyday use.",
    "Laptop": "A lightweight laptop featuring a powerful processor, ample storage, and a high-definition display, ideal for work and entertainment.",
    "Tablet": "A versatile tablet with a large screen, responsive touch interface, and long battery life, suitable for reading, browsing, and streaming.",
    "Smartwatch": "A stylish smartwatch that tracks your fitness, monitors your health, and keeps you connected with notifications and calls.",
    "Bluetooth Headphones": "Wireless headphones offering immersive sound quality, noise cancellation, and a comfortable fit for all-day wear.",
    "Wireless Earbuds": "Compact earbuds with touch controls, sweat resistance, and superior sound quality for an on-the-go lifestyle.",
    "Smart TV": "A smart television with built-in streaming apps, voice control, and high-definition picture quality for an enhanced viewing experience.",
    "Digital Camera": "A high-resolution digital camera with optical zoom, image stabilization, and various shooting modes for capturing memories.",
    "Portable Speaker": "A waterproof portable speaker with Bluetooth connectivity, delivering rich sound and deep bass for outdoor adventures.",
    "Power Bank": "A compact power bank with fast charging capabilities, ensuring your devices stay powered throughout the day.",
    "Electric Kettle": "A stainless steel electric kettle with rapid boiling technology, auto shut-off feature, and a sleek design.",
    "Hair Dryer": "A lightweight hair dryer with multiple heat settings, ionic technology, and a cool shot button for a smooth finish.",
    "Electric Toothbrush": "An electric toothbrush with sonic technology, multiple brushing modes, and a timer for effective oral care.",
    "Electric Shaver": "A cordless electric shaver with hypoallergenic blades, providing a close and comfortable shave.",
    "Microwave Oven": "A microwave oven with multiple cooking presets, defrost function, and a spacious interior for quick meal preparation.",
    "Refrigerator": "A double-door refrigerator with energy-efficient cooling, adjustable shelves, and a spacious freezer compartment.",
    "Washing Machine": "A front-load washing machine with multiple wash programs, energy efficiency, and a large capacity drum.",
    "Air Conditioner": "A split air conditioner with inverter technology, providing efficient cooling and low noise operation.",
    "Water Purifier": "A multi-stage water purifier with UV and RO filtration, ensuring safe and clean drinking water.",
    "Electric Fan": "A high-speed electric fan with oscillation, multiple speed settings, and a sleek design for optimal air circulation.",
    "Induction Cooktop": "A portable induction cooktop with touch controls, multiple power levels, and a sleek glass surface.",
    "Electric Pressure Cooker": "A programmable electric pressure cooker with multiple cooking presets, ensuring quick and easy meal preparation.",
    "Electric Grill": "An electric grill with non-stick plates, adjustable temperature control, and a drip tray for healthier cooking.",
    "Electric Rice Cooker": "A rice cooker with one-touch cooking, keep-warm function, and a non-stick inner pot for easy cleaning.",
    "Electric Juicer": "A centrifugal juicer with wide feeding tube, stainless steel blades, and anti-drip spout for fresh juice.",
    "Electric Mixer Grinder": "A powerful mixer grinder with multiple jars, stainless steel blades, and overload protection for versatile blending.",
    "Electric Iron": "A steam iron with ceramic soleplate, vertical steam function, and anti-drip system for wrinkle-free clothes.",
    "Electric Heater": "A compact electric heater with adjustable thermostat, overheat protection, and quiet operation for cozy warmth.",
    "Electric Blanket": "A soft electric blanket with multiple heat settings, auto shut-off feature, and machine-washable fabric.",
    "Electric Stove": "A portable electric stove with multiple burners, adjustable heat controls, and a compact design for easy storage.",
    "Electric Oven": "A countertop electric oven with convection cooking, multiple rack positions, and a timer for precise baking.",
    "Electric Chimney": "A wall-mounted electric chimney with powerful suction, LED lighting, and easy-to-clean filters for a smoke-free kitchen.",
    "Electric Geyser": "A high-efficiency electric geyser with fast heating, temperature control, and safety features for hot water.",
    "Electric Water Heater": "A tankless electric water heater with instant heating, energy-saving mode, and compact design for space-saving.",
    "Electric Cooker": "A multi-functional electric cooker with rice cooking, steaming, and slow cooking capabilities in one appliance.",
    "Electric Deep Fryer": "A deep fryer with adjustable temperature control, large capacity, and oil filtration system for crispy fried foods.",
    "Electric Sandwich Maker": "A sandwich maker with non-stick plates, adjustable browning control, and cool-touch handles for safe use.",
    "Electric Popcorn Maker": "A hot air popcorn maker with no oil required, producing healthy and fluffy popcorn in minutes.",
    "Electric Coffee Maker": "A programmable coffee maker with brew strength control, auto-shut off, and a reusable filter for fresh coffee.",
    "Electric Tea Maker": "An electric tea maker with temperature control, keep-warm function, and stainless steel infuser for perfect tea.",
    "Electric Milk Frother": "A milk frother with multiple frothing options, stainless steel whisk, and easy-clean design for creamy beverages."}
    
    color = random.choice([
    "Red", "Blue", "Black", "White", "Green", "Beige", "Ivory", "Lavender", "Fuchsia", "Carmine",
    "Viridian", "Mauve", "Baby Blue", "Emerald", "Turquoise", "Amber", "Coral", "Peach", "Magenta", "Crimson",
    "Aqua", "Indigo", "Olive", "Charcoal", "Maroon", "Sapphire", "Plum", "Teal", "Rose", "Lime",
    "Mustard", "Slate", "Copper", "Jade", "Periwinkle", "Cobalt", "Blush", "Mint", "Sunset Orange", "Seafoam"])
    
    product_sizes = {
    # Apparel & Accessories
    "Shirt": ["XS", "S", "M", "L", "XL", "XXL"],
    "Shoes": ["6", "7", "8", "9", "10", "11", "12"],
    "Watch": ["38mm", "40mm", "42mm", "44mm"],
    "Jacket": ["S", "M", "L", "XL", "XXL"],
    "Backpack": ["Small", "Medium", "Large"],
    "Sunglasses": ["One Size"],
    "Sweater": ["S", "M", "L", "XL"],
    "Kurta": ["S", "M", "L", "XL", "XXL"],
    "Jeans": ["28", "30", "32", "34", "36", "38"],
    "T-shirt": ["XS", "S", "M", "L", "XL", "XXL"],
    "Scarf": ["One Size"],
    "Hat": ["S", "M", "L"],
    "Sneakers": ["6", "7", "8", "9", "10", "11", "12"],
    "Boots": ["6", "7", "8", "9", "10", "11", "12"],
    "Blazer": ["S", "M", "L", "XL"],
    "Wallet": ["One Size"],
    "Gloves": ["S", "M", "L"],
    "Belt": ["28", "30", "32", "34", "36", "38", "40"],
    "Tie": ["One Size"],
    "Cufflinks": ["One Size"],
    "Chinos": ["28", "30", "32", "34", "36", "38"],
    "Cardigan": ["S", "M", "L", "XL"],
    "Pajamas": ["S", "M", "L", "XL"],
    "Shorts": ["S", "M", "L", "XL"],
    "Raincoat": ["S", "M", "L", "XL"],
    "Sweatshirt": ["S", "M", "L", "XL"],
    "Tracksuit": ["S", "M", "L", "XL"],
    "Vest": ["S", "M", "L", "XL"],
    "Overcoat": ["S", "M", "L", "XL"],
    "Sandals": ["6", "7", "8", "9", "10", "11"],
    "Flip-flops": ["6", "7", "8", "9", "10", "11"],
    "Beanie": ["One Size"],
    "Headband": ["One Size"],
    "Socks": ["S", "M", "L"],
    "Umbrella": ["Small", "Medium", "Large"],
    "Luggage": ["20\"", "24\"", "28\""],
    "Duffel Bag": ["Small", "Medium", "Large"],
    "Laptop Sleeve": ["13\"", "14\"", "15.6\""],

    # Electronics & Appliances
    "Smartphone": ["5.5\"", "6.1\"", "6.7\""],
    "Laptop": ["13\"", "14\"", "15.6\"", "17.3\""],
    "Tablet": ["7\"", "8\"", "10.2\"", "12.9\""],
    "Smartwatch": ["38mm", "40mm", "44mm", "46mm"],
    "Bluetooth Headphones": ["One Size"],
    "Wireless Earbuds": ["One Size"],
    "Smart TV": ["32\"", "43\"", "50\"", "55\"", "65\"", "75\"", "85\""],
    "Digital Camera": ["APS-C", "Full Frame"],
    "Portable Speaker": ["Small", "Medium", "Large"],
    "Power Bank": ["5,000mAh", "10,000mAh", "20,000mAh"],
    "Electric Kettle": ["1L", "1.5L", "2L"],
    "Hair Dryer": ["One Size"],
    "Electric Toothbrush": ["One Size"],
    "Electric Shaver": ["One Size"],
    "Microwave Oven": ["20L", "25L", "30L", "32L", "35L"],
    "Refrigerator": ["200L", "300L", "500L"],
    "Washing Machine": ["6kg", "8kg", "10kg"],
    "Air Conditioner": ["1 Ton", "1.5 Ton", "2 Ton"],
    "Water Purifier": ["5L", "7L", "10L"],
    "Electric Fan": ["One Size"],
    "Induction Cooktop": ["One Size"],
    "Electric Pressure Cooker": ["One Size"],
    "Electric Grill": ["One Size"],
    "Electric Rice Cooker": ["One Size"],
    "Electric Juicer": ["One Size"],
    "Electric Mixer Grinder": ["One Size"],
    "Electric Iron": ["One Size"],
    "Electric Heater": ["One Size"],
    "Electric Blanket": ["Single", "Double", "Queen", "King"],
    "Electric Stove": ["Single Burner", "Double Burner"],
    "Electric Oven": ["18L", "28L", "35L"],
    "Electric Chimney": ["60cm", "90cm"],
    "Electric Geyser": ["10L", "15L", "25L"],
    "Electric Water Heater": ["3kW", "5kW"],
    "Electric Cooker": ["1L", "1.5L", "2L"],
    "Electric Deep Fryer": ["2L", "3L", "4L"],
    "Electric Sandwich Maker": ["One Size"],
    "Electric Popcorn Maker": ["One Size"],
    "Electric Coffee Maker": ["4 Cup", "6 Cup", "12 Cup"],
    "Electric Tea Maker": ["0.5L", "1L", "1.5L"],
    "Electric Milk Frother": ["One Size"]
    }

    # Get the Product details (name, description & details)
    product_details = get_product_detail(product_name, product_descriptions, product_sizes, color)
    product_description = product_details["Product_Description"]
    product_size = product_details["Product_Size"]
    product_color = product_details["Product_color"]
    
    
    # PRODUCTS
    avg_inventory = random.randint(1, 100)
    #c_avg_inventory = product_inventory_adjust(avg_inventory)
    
    days_since_creation = random.randint(1, 360)
    #c_days_since_creation = product_age_adjust(days_since_creation)
    
    images_count = random.randint(0, 6)
    #c_images_count = produt_image_adjust(images_count)
    
    product_type = random.choice(["seasonal", "regular"])
    #c_product_type = product_type_adjust(product_type)
    
    product_tag = random.choice(["winter", "summer", "spring", "autumn", "electronics", "luxury", "casual"])
    #c_product_tag = product_tag_adjust(product_tag)
    
    product_price = random.randint(199, 9000)
    #c_product_price = avg_price_adjust(product_price)
    
    # Dashboard Data Clint Minimum Price
    min_price_client, min_price_dicount = get_min_price_from_product_price(product_price)


    # USER
    orders = random.randint(0, 12)
    #c_orders = user_order_score(orders)
    
    spent = random.randint(50, 10000)
    #c_spent = user_spending_adjust(spent)
    
    refunded = random.randint(0, 600)
   # c_refunded = user_refund_penalty(refunded, spent)
    
    days_since_last = random.randint(1, 400)
    #c_days_since_last = user_recency_bonus(days_since_last)

    #c_verified = user_email_verify_adjust(user_email)

    # User, Product, Bargain Scores.
    user_score = calculate_user_score(orders, refunded, spent, days_since_last, user_email)
    product_score = calculate_product_score(days_since_creation, avg_inventory, images_count, product_type, product_tag, product_price)
    bargain_score = random.randint(30, 100)

# measure ments taken in current_round_count iteration     
    # Round Counts (user, product, bargain)
    max_round_count = calculate_max_rounds(user_score, product_score, bargain_score)
    # get lower bound based on max round_count
    lower_bound_cr = get_lower_bound(max_round_count)
    current_round = random.randint(lower_bound_cr, max_round_count)
    
    adjusted_discount = adjust_max_discount(min_price_dicount, user_score, product_score, bargain_score)
    round_counts_with_discounts = generate_all_round_discounts(max_round_count, adjusted_discount)
    
    
## current round, over all round and discount logical pipeline 
    previous_discount_given, user_req = get_previous_round(current_round, round_counts_with_discounts)
    
    # -----------------------------------------------------------------------------------

    # Get discount for current round
    current_discount = get_discount_for_current_round(current_round, round_counts_with_discounts)
    
    discounted_price_current_round = round(get_original_price_from_discounts(product_price, current_discount), 2)
    
    if max_round_count == current_round:
        response_message = (
            f"Bot: thats way much higer {user_req}% since you have came this much far i can lower the price from myside let's see what i can do, how about {discounted_price_current_round}.Rs which is upto {current_discount}%. this is my final offer and a compact deal, i can get you now itself"
        )
    else:
        response_message = (
            f"Bot: That’s quite a high ask at {user_req}%. But since you've come this far, I can drop the price to {discounted_price_current_round} INR, which gives you {current_discount}% off."
        )

    Data_from_data_base = (
        f"User Info:\n"
        f"user_name = {user_name}, email = {user_email}, "
        f"orders = {orders}, spent = {spent}, email_verified = {user_email}, refunds = {refunded}, days_since_last = {days_since_last}\n\n"

        f"Product Info:\n"
        f"product_name = {product_name}, product_description = {product_description}, product_color = {product_color}, product_size = {product_size}, "
        f"product_price = {product_price} INR, min_price_client = {min_price_client} INR, avg_inventory = {avg_inventory}, "
        f"product_tag = {product_tag}, product_type = {product_type}, images_count = {images_count}, days_since_creation = {days_since_creation}\n\n"   
    )

    user = (
#        f"User: along with that text {previous_roudn_discount} and {user_req_discount} prep this U_discoun from the previous discount ?"
        f"User: {previous_discount_given}% is less it seems is that all you can offer,ok lets say if you can make it to {user_req} thats a deal"
    )

    response = response_message


    n_user = normalize_text(user)
    n_response = normalize_text(response)
    n_Data_from_data_base = normalize_text(Data_from_data_base)

    return {
       "Live_data_from_frontend": n_Data_from_data_base,
        "user": n_user,
        "response": n_response,
        "metadata": {
            # User
            "user_email_id": user_email,
            "user_score": user_score,
            # Bargain 
            "bargain_score": bargain_score,
            # Product
            "product_price": product_price,
            "product_score": product_score,
            "admin_max_discount_from_min_price": min_price_dicount,
            "system_max_discount": adjusted_discount,
            "max_round_count": max_round_count,
            #"lower_bound": lower_bound_cr,
            "current_round": current_round,
            "previous_round_discount_given" : previous_discount_given,
            "user_req": user_req,
            "current_discount": current_discount,
            "current_discount_price": discounted_price_current_round,
            "round_discounts": round_counts_with_discounts
        }
    }

In [ ]:
# file save

import json
from tqdm import trange  # Install with `pip install tqdm` if needed

# Dataset Generator with progress bar and error handling
def generate_dataset(n=100, filename="/mnt/data/bargenix_single_turn_full_500.jsonl"):
    successful = 0
    with open(filename, "w") as f:
        for i in trange(n, desc="Generating Samples"):
            try:
                sample = generate_scenario()
                f.write(json.dumps(sample) + "\n")
                successful += 1
            except Exception as e:
                print(f"⚠️ Error generating sample {i + 1}: {e}")
    print(f"✅ Successfully generated {successful}/{n} samples → {filename}")


In [12]:
# generate_dataset() for Jupyter (no file saving)

import json
from tqdm import trange  # For optional progress bar display

def n_generate_datasets(n=50):
    dataset = []
    for i in trange(n, desc="Generating Samples"):
        sample = generate_scenario()
        dataset.append(sample)
        except Exception as e:
            print(f"⚠️ Error generating sample {i + 1}: {e}")
    
    # Print as formatted JSON
    print(json.dumps(dataset, indent=2))
    
    return dataset  # Optional: Return the dataset if you want to work with it later


In [15]:
data = n_generate_datasets(n=5)

Generating Samples: 100%|████████████████████████████████████████████████████████████████████████| 5/5 [00:00<?, ?it/s]

[
  {
    "Live_data_from_frontend": "User Info:\nuser_name = Shanaya, email = shanaya@gmail.com, orders = 3, spent = 5649, email_verified = shanaya@gmail.com, refunds = 228, days_since_last = 177\n\nProduct Info:\nproduct_name = Electric Fan, product_description = A high-speed electric fan with oscillation, multiple speed settings, and a sleek design for optimal air circulation., product_color = Blue, product_size = One Size, product_price = 1355 INR, min_price_client = 759 INR, avg_inventory = 94, product_tag = summer, product_type = seasonal, images_count = 1, days_since_creation = 132\n\n",
    "user": "User: 19% is less it seems is that all you can offer,ok lets say if you can make it to 31 thats a deal",
    "response": "Bot: Thats quite a high ask at 31%. But since you've come this far, I can drop the price to 1070.45 INR, which gives you 21% off.",
    "metadata": {
      "user_email_id": "shanaya@gmail.com",
      "user_score": 25.457662169814718,
      "bargain_score": 69,
  